In [113]:
from functools import cache
from queue import PriorityQueue as PQ


import networkx as nx

In [66]:
INPUT_TEST_ = """###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############"""
MAP_TEST = [list(s) for s in INPUT_TEST_.split('\n')]

INPUT_TEST_1_ = """#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################"""
MAP_TEST_1 = [list(s) for s in INPUT_TEST_1_.split('\n')]

with open('d16_in.txt', 'r') as f:
    INPUT_ = f.read()
MAP = [list(s) for s in INPUT_.split('\n')]

In [97]:
def get_start(map_):
    for i in range(len(map_)):
        for j in range(len(map_[0])):
            if map_[i][j] == 'S':
                return (i, j)
    return (-1, -1)

def get_end(map_):
    for i in range(len(map_)):
        for j in range(len(map_[0])):
            if map_[i][j] == 'E':
                return (i, j)
    return (-1, -1)

def get_nbh_cost(nb_t, map_):
    v, d = nb_t
    i, j = v
    match d:
        case '^':
            nb_c = [((i-1, j), '^', 1), ((i, j), '>', 1000), ((i, j), '<', 1000)]
        case 'v':
            nb_c = [((i+1, j), 'v', 1), ((i, j), '>', 1000), ((i, j), '<', 1000)]
        case '>':
            nb_c = [((i, j+1), '>', 1), ((i, j), '^', 1000), ((i, j), 'v', 1000)]
        case '<':
            nb_c = [((i, j-1), '<', 1), ((i, j), '^', 1000), ((i, j), 'v', 1000)]
    return [
        nb for nb in nb_c
        if map_[nb[0][0]][nb[0][1]] != '#'
    ]

Part 1

In [98]:
BIG_NUM = (1 << 31) -1

In [94]:
def dij(map_):

    dist = {((i, j), d): BIG_NUM
        for i in range(1, len(map_)-1)
        for j in range(1, len(map_[0])-1)
        for d in "^v><"}
    pred = {((i, j), d): None
        for i in range(1, len(map_)-1)
        for j in range(1, len(map_[0])-1)
        for d in "^v><"}

    Q = PQ()
    s_ = get_start(map_)
    s = (s_, '>')
    dist[s] = 0

    Q.put((0, s))
    for v in dist:
        if v != s:
            Q.put((BIG_NUM, v))

    e_ = get_end(map_)
    ends = [(e_, d) for d in '^v><']

    mins = []
    while not Q.empty():
        dist_u, u = Q.get()
        if u in ends:
            mins.append((dist_u, u))
        if len(mins) == 4:
            break
        for nb_t in get_nbh_cost(u, map_):
            v, cost = (nb_t[0], nb_t[1]), nb_t[2]
            alt = dist_u + cost
            if alt < dist[v]:
                pred[v] = u
                dist[v] = alt
                Q.put((alt, v))
    return mins, pred, dist

In [109]:
map_ = MAP_TEST

mins=[]
mins, pred, dist = dij(map_)
mins

[(7036, ((1, 13), '^')),
 (8036, ((1, 13), '<')),
 (8036, ((1, 13), '>')),
 (9036, ((1, 13), 'v'))]

Part 2

In [112]:
fwd = {p: [] for p in pred.values()}
for k, v in pred.items():
    fwd[v].append(k)

@cache
def count(curr, end):
    if curr == end:
        return 1
    if not fwd[curr]:
        return 0
    total = 0
    for n in fwd[curr]:
        total += count(n, end)
    return total

NameError: name 'cache' is not defined

In [110]:
G = nx.Graph()

st_ = get_start(map_)
start = (st_, '>')

end = min(mins)[1]

for i in range(1, len(map_)-1):
    for j in range(1, len(map_[0])-1):
        for d in "><^v":
            u = ((i, j), d)
            for tup in get_nbh_cost(u, map_):
                v = (tup[0], tup[1])
                weight = tup[2]
                G.add_edge(u, v, weight=weight)

count = 0
for p in nx.all_shortest_paths(G, source=start, target=end, weight=weight):
    count += 1
count

3072